In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
import os


CUDA_LAUNCH_BLOCKING=1
%matplotlib inline

# Download census-income dataset

In [2]:
torch.__version__

'1.10.0+cu102'

In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
dataset_name = 'census-income'
out = Path(os.getcwd()+'/data/'+dataset_name+'.csv')

In [4]:
out.parent.mkdir(parents=True, exist_ok=True)
if out.exists():
    print("File already exists.")
else:
    print("Downloading file...")
    wget.download(url, out.as_posix())

File already exists.


# Load data and split

In [5]:
train = pd.read_csv(out)
target = ' <=50K'
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

# Simple preprocessing

Label encode categorical features and fill empty cells.

In [6]:
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)

39 73
 State-gov 9
 Bachelors 16
 13 16
 Never-married 7
 Adm-clerical 15
 Not-in-family 6
 White 5
 Male 2
 2174 119
 0 92
 40 94
 United-States 42
 <=50K 2
Set 3


In [7]:
# check that pipeline accepts strings
train.loc[train[target]==0, target] = "wealthy"
train.loc[train[target]==1, target] = "not_wealthy"

# Define categorical features for categorical embeddings

In [8]:
unused_feat = ['Set']

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]


# Network parameters

In [9]:
tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetClassifier(**tabnet_params
                      )

/work/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


# Training

In [10]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

In [11]:
max_epochs = 100 if not os.getenv("CI", False) else 2

In [12]:
# This illustrates the warm_start=False behaviour
save_history = []
for _ in range(2):
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['auc'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False
    )
    save_history.append(clf.history["valid_auc"])
    
assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 0.66728 | train_auc: 0.75729 | valid_auc: 0.7552  |  0:00:01s
epoch 1  | loss: 0.51304 | train_auc: 0.82287 | valid_auc: 0.82513 |  0:00:02s
epoch 2  | loss: 0.46881 | train_auc: 0.84484 | valid_auc: 0.8424  |  0:00:03s
epoch 3  | loss: 0.4501  | train_auc: 0.86782 | valid_auc: 0.86474 |  0:00:05s
epoch 4  | loss: 0.42781 | train_auc: 0.8833  | valid_auc: 0.88173 |  0:00:06s
epoch 5  | loss: 0.41078 | train_auc: 0.89628 | valid_auc: 0.89008 |  0:00:07s
epoch 6  | loss: 0.39827 | train_auc: 0.90308 | valid_auc: 0.89607 |  0:00:09s
epoch 7  | loss: 0.391   | train_auc: 0.90791 | valid_auc: 0.89755 |  0:00:10s
epoch 8  | loss: 0.3836  | train_auc: 0.91095 | valid_auc: 0.90258 |  0:00:11s
epoch 9  | loss: 0.36724 | train_auc: 0.9144  | valid_auc: 0.90746 |  0:00:12s
epoch 10 | loss: 0.36825 | train_auc: 0.91749 | valid_auc: 0.91127 |  0:00:14s
epoch 11 | loss: 0.35866 | train_auc: 0.91651 | valid_auc: 0.91007 |  0:00:15s
epoch 12 | loss: 0.35291 | train_auc: 0.9157  | vali

/work/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.66728 | train_auc: 0.75729 | valid_auc: 0.7552  |  0:00:01s
epoch 1  | loss: 0.51304 | train_auc: 0.82287 | valid_auc: 0.82513 |  0:00:02s
epoch 2  | loss: 0.46881 | train_auc: 0.84484 | valid_auc: 0.8424  |  0:00:03s
epoch 3  | loss: 0.4501  | train_auc: 0.86782 | valid_auc: 0.86474 |  0:00:05s
epoch 4  | loss: 0.42781 | train_auc: 0.8833  | valid_auc: 0.88173 |  0:00:06s
epoch 5  | loss: 0.41078 | train_auc: 0.89628 | valid_auc: 0.89008 |  0:00:07s
epoch 6  | loss: 0.39827 | train_auc: 0.90308 | valid_auc: 0.89607 |  0:00:09s
epoch 7  | loss: 0.391   | train_auc: 0.90791 | valid_auc: 0.89755 |  0:00:10s
epoch 8  | loss: 0.3836  | train_auc: 0.91095 | valid_auc: 0.90258 |  0:00:11s
epoch 9  | loss: 0.36724 | train_auc: 0.9144  | valid_auc: 0.90746 |  0:00:13s
epoch 10 | loss: 0.36825 | train_auc: 0.91749 | valid_auc: 0.91127 |  0:00:14s
epoch 11 | loss: 0.35866 | train_auc: 0.91651 | valid_auc: 0.91007 |  0:00:15s
epoch 12 | loss: 0.35291 | train_auc: 0.9157  | vali

/work/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
# plot losses
plt.plot(clf.history['loss'])

In [ ]:
# plot auc
plt.plot(clf.history['train_auc'])
plt.plot(clf.history['valid_auc'])

In [ ]:
# plot learning rates
plt.plot(clf.history['lr'])

## Predictions

In [ ]:
preds = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)


preds_valid = clf.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=preds_valid[:,1], y_true=y_valid)

print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")

In [ ]:
# check that best weights are used
assert np.isclose(valid_auc, np.max(clf.history['valid_auc']), atol=1e-6)

In [ ]:
clf.predict(X_test)

# Save and load Model

In [ ]:
# save tabnet model
saving_path_name = "./tabnet_model_test_1"
saved_filepath = clf.save_model(saving_path_name)

In [ ]:
# define new model with basic parameters and load state dict weights
loaded_clf = TabNetClassifier()
loaded_clf.load_model(saved_filepath)

In [ ]:
loaded_preds = loaded_clf.predict_proba(X_test)
loaded_test_auc = roc_auc_score(y_score=loaded_preds[:,1], y_true=y_test)

print(f"FINAL TEST SCORE FOR {dataset_name} : {loaded_test_auc}")

In [ ]:
assert(test_auc == loaded_test_auc)

In [ ]:
loaded_clf.predict(X_test)

# Global explainability : feat importance summing to 1

In [ ]:
clf.feature_importances_

# Local explainability and masks

In [ ]:
explain_matrix, masks = clf.explain(X_test)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,20))

for i in range(3):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")


# XGB

In [ ]:
from xgboost import XGBClassifier

clf_xgb = XGBClassifier(max_depth=8,
    learning_rate=0.1,
    n_estimators=1000,
    verbosity=0,
    silent=None,
    objective='binary:logistic',
    booster='gbtree',
    n_jobs=-1,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.7,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=0,
    seed=None,)

clf_xgb.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=40,
        verbose=10)

In [ ]:
preds = np.array(clf_xgb.predict_proba(X_valid))
valid_auc = roc_auc_score(y_score=preds[:,1], y_true=y_valid)
print(valid_auc)

preds = np.array(clf_xgb.predict_proba(X_test))
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)
print(test_auc)